In [11]:
from pyspark import SparkContext
sc=SparkContext()

In [12]:
import re
import datetime
import sys
import os


In [50]:
data = sc.parallelize([("userid1","This is a tweet","tweets", "timestamp1", "source1"),("userid2","content2","content_type2", "timestamp2", "source2")])

In [51]:
'''TWITTER_PATH = 'Twitter.txt'
DATAFILE_PATTERN = '^(.+),"(.*)",(.+),(.*)'


def TWITTER_parser(filename):
    
    return (sc
            .textFile(filename, 4, 1)
            .cache())


DataRDD = Twitter_parser(TWITTER_PATH)'''
DataRDD = data
TweetsRDD = DataRDD.filter(lambda (a,b,c,d,e): c=='tweets')
RetweetCountsRDD = DataRDD.filter(lambda (a,b,c,d,e): c=='retweet_counts')
ReplyCountsRDD = DataRDD.filter(lambda (a,b,c,d,e): c=='reply_counts')
LikeCountsRDD = DataRDD.filter(lambda (a,b,c,d,e): c=='like_counts')

In [83]:
def removeQuotes(s):
    """ Remove quotation marks from an input string
    Args:
        s (str): input string that might have the quote "" characters
    Returns:
        str: a string without the quote characters
    """
    return ''.join(i for i in s if i!='"')


In [84]:
split_regex = r'[a-zA-Z0-9_]+'

def simpleTokenize(string):
    """ A simple implementation of input string tokenization
    Args:
        string (str): input string
    Returns:
        list: a list of tokens
    """
    string = re.sub(r'([^\s\w(?<_)])+', ' ',string.lower())
    return re.findall(split_regex, string)

def tokenize(string):
    """ An implementation of input string tokenization that excludes stopwords
    Args:
        string (str): input string
    Returns:
        list: a list of tokens without stopwords
    """
    
    def remove_if_not_substring(l1, l2):
        return [i for i in l1 if not any(j == i for j in list(l2))]
    string = remove_if_not_substring(simpleTokenize(string), stopwords)
    return string

In [85]:
baseDir = os.path.join('data')
inputPath = os.path.join('cs100', 'lab3')
STOPWORDS_PATH = 'stopwords.txt'
stopfile = os.path.join(baseDir, inputPath, STOPWORDS_PATH)
stopwords = set(sc.textFile(stopfile).collect())

In [86]:
TweetsTokenizedRDD = TweetsRDD.map(lambda (a,b,c,d,e): (a,tokenize(b),c,d,e))

In [87]:
def tf(tokens):
    """ Compute TF
    Args:
        tokens (list of str): input list of tokens from tokenize
    Returns:
        dictionary: a dictionary of tokens to its TF values
    """
    total_len = float(len(tokens))
    dictionary = {}
    for x in tokens:
        if x in dictionary.keys():
            dictionary[x] = dictionary[x]+1/total_len
        else:
            dictionary[x] = 1/total_len
    return dictionary ##dict(zip(tokens,[float(len(x))/total_len for x in tokens]))

In [88]:
def idfs(corpus):
    """ Compute IDF
    Args:
        corpus (RDD): input corpus
    Returns:
        RDD: a RDD of (token, IDF value)
    
    N = float(len(corpus.map(lambda x:x).collect()))
    uniqueTokens = corpus.flatMap(lambda x: [(y,x[0]) for y in x[1]]).distinct()
    tokenCountPairTuple = uniqueTokens.map(lambda x: (x[0],1))
    tokenSumPairTuple = tokenCountPairTuple.foldByKey(0, lambda a,b:a+b)
    return (tokenSumPairTuple.map(lambda x: (x[0], N/x[1])))"""
    N = corpus.count()
    uniqueTokens = corpus.map(lambda ut: set(ut[1]))
    tokenCountPairTuple = uniqueTokens.flatMap(lambda strg: [(key, 1) for key in strg]).reduceByKey(lambda x, y: x + y)
    tokenSumPairTuple = tokenCountPairTuple.map(lambda duple: (duple[0], 1. * N / duple[1]))
    return (tokenSumPairTuple)

In [92]:
TweetsRecTok = TweetsTokenizedRDD.map(lambda (a,b,c,d,e):b)
idf = idfs(TweetsRecTok)
print idfs

TypeError: count() takes exactly one argument (0 given)

In [93]:
idf.count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 38.0 failed 1 times, most recent failure: Lost task 0.0 in stage 38.0 (TID 36, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/worker.py", line 101, in main
    process()
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/worker.py", line 96, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/rdd.py", line 2252, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/rdd.py", line 2252, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/rdd